In [ ]:
# Load the functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report



import matplotlib.pyplot as plt
import seaborn as sns

1. Bank Marketing Data
Download the bank direct marketing dataset banking.csv to train a logistic regression (scikit-learn) predicting if a client will subscribe (1 or 0) to a term deposit (variable called ‘y’).


In [ ]:
# Load the banking dataset
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('drive/MyDrive/04_MachineLearning/banking.csv')
df.head(10)

Mounted at /content/drive


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1
5,30,management,divorced,basic.4y,no,yes,no,cellular,jul,tue,...,8,999,0,nonexistent,1.4,93.918,-42.7,4.961,5228.1,0
6,37,blue-collar,married,basic.4y,no,yes,no,cellular,may,thu,...,1,999,0,nonexistent,-1.8,92.893,-46.2,1.327,5099.1,0
7,39,blue-collar,divorced,basic.9y,no,yes,no,cellular,may,fri,...,1,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,0
8,36,admin.,married,university.degree,no,no,no,cellular,jun,mon,...,1,3,1,success,-2.9,92.963,-40.8,1.266,5076.2,1
9,27,blue-collar,single,basic.4y,no,yes,no,cellular,apr,thu,...,2,999,1,failure,-1.8,93.075,-47.1,1.410,5099.1,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp_var_rate    41188 non-null  float64
 16  cons_price_idx  41188 non-null  float64
 17  cons_conf_idx   41188 non-null 

In [ ]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp_var_rate      0
cons_price_idx    0
cons_conf_idx     0
euribor3m         0
nr_employed       0
y                 0
dtype: int64

Create a ML pipeline taking in account the following points:

1.1. Split data into a train and test set.

In [ ]:
X= df.drop(columns=['y','duration'])  #regarding data leakeage, I deleted 'duration' column. 
y=df['y']

X.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx',
       'euribor3m', 'nr_employed'],
      dtype='object')

In [ ]:
y.value_counts()

# Shows imbalances between the classes of the data

0    36548
1     4640
Name: y, dtype: int64

In [ ]:
# Splitting the data for the Logistic Regression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train.shape, X_test.shape

((28831, 19), (12357, 19))

1.2. Selecting the feature and target columns. Deleting feature that would introduce data leakage.

In [ ]:
# Define feature / target columns

categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_features = X_train.select_dtypes(include=['int', 'float']).columns.tolist()

categorical_features, numeric_features

(['job',
  'marital',
  'education',
  'default',
  'housing',
  'loan',
  'contact',
  'month',
  'day_of_week',
  'poutcome'],
 ['age',
  'campaign',
  'pdays',
  'previous',
  'emp_var_rate',
  'cons_price_idx',
  'cons_conf_idx',
  'euribor3m',
  'nr_employed'])

In [ ]:
categorical_transformer = Pipeline(steps=[
                                          ("cat_imputer", SimpleImputer(strategy='constant',
                                                                        fill_value='Not Available')),
                                          ("onehot", OneHotEncoder(handle_unknown="ignore"))
                                          ])

In [ ]:
numeric_transformer = Pipeline(steps=[
                                      ("knn_imputer", KNNImputer(n_neighbors=5)),
                                      ("scaler", StandardScaler())
                                      ])

In [ ]:
preprocessor = ColumnTransformer(transformers=[
                                               ("num", numeric_transformer, 
                                                       numeric_features),
                                               ("cat", categorical_transformer, 
                                                       categorical_features)
                                               ])

1.3. Use class_weight = ‘balanced’, as the dataset is unbalanced

In [ ]:
lr_model = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced')

In [ ]:
pipeline_lr = Pipeline([("col_transformer", preprocessor), 
                            ("estimator", lr_model)])

In [ ]:
pipeline_lr.fit(X_train, y_train)

y_pred = pipeline_lr.predict(X_test)

In [ ]:
class_labels = pipeline_lr.named_steps['estimator'].classes_
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

,0,1
0,9374,1591
1,534,858


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     10965
           1       0.35      0.62      0.45      1392

    accuracy                           0.83     12357
   macro avg       0.65      0.74      0.67     12357
weighted avg       0.88      0.83      0.85     12357



1.4. k-fold cross-validation to choose between L1- and L2-penalty or no penalty (specify penalty and C, logistic regression documentation). Use F1 score as a metric.
Once I have chosen the best model by cross-validation, evaluate its performance on the test set.

In [ ]:
# Improving logistic regression model: defining penalty as l1 with lower C level
lr_model1 = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced', penalty='l1', C=0.1 )
pipeline_lr1 = Pipeline([("col_transformer", preprocessor), 
                            ("estimator", lr_model1)])
pipeline_lr1.fit(X_train, y_train)

y_pred = pipeline_lr1.predict(X_test)
class_labels = pipeline_lr1.named_steps['estimator'].classes_
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

,0,1
0,9350,1615
1,531,861


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     10965
           1       0.35      0.62      0.45      1392

    accuracy                           0.83     12357
   macro avg       0.65      0.74      0.67     12357
weighted avg       0.88      0.83      0.85     12357



In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline_lr1, X_train, y_train, cv=5, scoring='f1')
scores.mean()

0.4547091709261896

In [ ]:
# Improving logistic regression model: defining penalty as l1 and higher C level 
lr_model1 = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced', penalty='l1', C=1 )
pipeline_lr1 = Pipeline([("col_transformer", preprocessor), 
                            ("estimator", lr_model1)])
pipeline_lr1.fit(X_train, y_train)

y_pred = pipeline_lr1.predict(X_test)
class_labels = pipeline_lr1.named_steps['estimator'].classes_
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

,0,1
0,9372,1593
1,535,857


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     10965
           1       0.35      0.62      0.45      1392

    accuracy                           0.83     12357
   macro avg       0.65      0.74      0.67     12357
weighted avg       0.88      0.83      0.85     12357



In [ ]:
# Improving logistic regression model: defining penalty as l1 and higher C level 
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline_lr1, X_train, y_train, cv=5, scoring='f1')
scores.mean()

0.4546163043175456

In [ ]:
# Logistic Regression model with Penalty level l2 and lower C regulation
lr_model2 = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced', penalty='l2', C=0.1 )
pipeline_lr2 = Pipeline([("col_transformer", preprocessor), 
                            ("estimator", lr_model2)])
pipeline_lr2.fit(X_train, y_train)

y_pred = pipeline_lr2.predict(X_test)
class_labels = pipeline_lr2.named_steps['estimator'].classes_
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

,0,1
0,9334,1631
1,528,864


In [ ]:
 print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     10965
           1       0.35      0.62      0.44      1392

    accuracy                           0.83     12357
   macro avg       0.65      0.74      0.67     12357
weighted avg       0.88      0.83      0.85     12357



In [ ]:
# Improving logistic regression model: defining penalty as l2 and Lower C level 
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline_lr2, X_train, y_train, cv=5, scoring='f1')
scores.mean()

0.4524166216257538

In [ ]:
# Logistic Regression model with Penalty level l2 and higher C regulation
lr_model2 = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced', penalty='l2', C=1 )
pipeline_lr2 = Pipeline([("col_transformer", preprocessor), 
                            ("estimator", lr_model2)])
pipeline_lr2.fit(X_train, y_train)

y_pred = pipeline_lr2.predict(X_test)
class_labels = pipeline_lr2.named_steps['estimator'].classes_
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

,0,1
0,9374,1591
1,534,858


In [ ]:
 print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     10965
           1       0.35      0.62      0.45      1392

    accuracy                           0.83     12357
   macro avg       0.65      0.74      0.67     12357
weighted avg       0.88      0.83      0.85     12357



In [ ]:
# Improving logistic regression model: defining penalty as l2 and Lower C level 
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline_lr2, X_train, y_train, cv=5, scoring='f1')
scores.mean()

0.454341048287647

In [ ]:
# Logistic Regression model : without penalty , with low C regulation
lr_model3 = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced', penalty=None, C=0.1 )
pipeline_lr3 = Pipeline([("col_transformer", preprocessor), 
                            ("estimator", lr_model2)])
pipeline_lr3.fit(X_train, y_train)

y_pred = pipeline_lr3.predict(X_test)
class_labels = pipeline_lr3.named_steps['estimator'].classes_
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

,0,1
0,9374,1591
1,534,858


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     10965
           1       0.35      0.62      0.45      1392

    accuracy                           0.83     12357
   macro avg       0.65      0.74      0.67     12357
weighted avg       0.88      0.83      0.85     12357



In [ ]:
# Logistic Regression model : without penalty , with low C regulation
scores = cross_val_score(pipeline_lr3, X_train, y_train, cv=5, scoring='f1')
scores.mean()

0.454341048287647

In [ ]:
# Logistic Regression model : without penalty , with low C regulation
lr_model3 = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced', penalty=None, C=1 )
pipeline_lr3 = Pipeline([("col_transformer", preprocessor), 
                            ("estimator", lr_model2)])
pipeline_lr3.fit(X_train, y_train)

y_pred = pipeline_lr3.predict(X_test)
class_labels = pipeline_lr3.named_steps['estimator'].classes_
pd.DataFrame(confusion_matrix(y_test, y_pred), 
             columns=class_labels, index=class_labels)

,0,1
0,9374,1591
1,534,858


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     10965
           1       0.35      0.62      0.45      1392

    accuracy                           0.83     12357
   macro avg       0.65      0.74      0.67     12357
weighted avg       0.88      0.83      0.85     12357



Using a classic Machine Learning model, I predicted if a client subscribe (1 or 0) depending on the deposite. Overall, the accuracy of predicting the clients who are not going to subscribe was higher than the clients who are going to susbscribe, because there were more cases of the client who will not subscribe. Consequently, the present project showed the use case of the classical machine learning model for checking the impact of the marketing on customer subscription. 